<a href="https://colab.research.google.com/github/yardsale8/DSCI_210_R_notebooks/blob/main/lecture_6_5_making_two_way_tables_in_R.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Grouping and Aggregating in R

In [1]:
library(tidyverse)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [2]:
births <- read.csv('https://github.com/WSU-DataScience/DSCI_210_R_notebooks/raw/main/data/NC_Birth.csv')
head(births)

,Father.Minority,Father.Age,Father.Years.of.Education,Mother.Minority,Mother.Age,Mother.Years.of.Education,Marital.Status,Mother.Smoker,Mother.Avg.Cigs,Mother.Weight.Gain,⋯,Month.of.Birth,Day.of.Birth,Gender,Number.of.Prenatal.Visits,Kotelchuck.Index,Gestational.Age,Birth.Weight.Grams,Low.Birth,Apgar.Score.One.Minute,Apgar.Score.Five.Minutes
,<chr>,<int>,<int>,<chr>,<int>,<int>,<chr>,<chr>,<int>,<int>,⋯,<int>,<int>,<chr>,<chr>,<chr>,<int>,<int>,<chr>,<int>,<int>
1,Nonwhite,50,12,White,24,15,Unmarried,N,0,50,⋯,2,23,Female,10,3_Adequate,38,2866,N,9,9
2,White,19,9,White,18,9,Unmarried,Y,23,35,⋯,1,25,Female,9,3_Adequate,35,3121,N,8,9
3,White,37,17,White,35,17,Married,N,0,24,⋯,1,25,Female,20,4_AdequatePlus,38,2667,N,8,9
4,,NA,NA,Nonwhite,22,14,Unmarried,N,0,33,⋯,11,10,Male,15,4_AdequatePlus,38,2696,N,8,9
5,Nonwhite,39,11,Nonwhite,31,16,Unmarried,N,0,30,⋯,5,18,Female,12,1_Inadequate,38,2979,N,4,9
6,White,20,11,Nonwhite,19,12,Unmarried,N,0,10,⋯,11,6,Male,10,4_AdequatePlus,36,3036,N,9,9


# Outline

* Simple Aggregation
* Grouped Aggregation

# Aggregation

### Simple aggregation

<img src="https://github.com/yardsale8/DSCI_210_R_notebooks/blob/main/img/simple_aggregation.png?raw=1" width=600>

#### Beware of `NA`!

`NA + 1 == NA` in R

In [3]:
(births
 %>% summarize(avg_father_educ = mean(Father.Years.of.Education))
 )

avg_father_educ
<dbl>
NA


#### Option 1 -- Filter out incomplete rows

In [4]:
(births
 %>% filter(complete.cases(Father.Years.of.Education))
 %>% summarize(avg_father_educ = mean(Father.Years.of.Education))
 )

avg_father_educ
<dbl>
12.79394


#### Option 2 -- set `na.rm=TRUE`

In [5]:
(births
 %>% summarize(avg_father_educ = mean(Father.Years.of.Education, na.rm = TRUE))
 )

avg_father_educ
<dbl>
12.79394


### Grouped aggregation

<img src="https://github.com/yardsale8/DSCI_210_R_notebooks/blob/main/img/group_and_aggregate.png?raw=1" width=600>

#### One grouping variable

Using `group_by`

In [10]:
(births
 %>% group_by(Gender)
 %>% summarize(avg_wt = mean(Birth.Weight.Grams,
                             na.rm = TRUE)
              )
 )

Gender,avg_wt
<chr>,<dbl>
Female,3203.117
Male,3279.816


Using `.by` argument in `summarise`

In [11]:
(births
 %>% summarize(avg_wt = mean(Birth.Weight.Grams,
                             na.rm = TRUE),
              .by = Gender)


 )

Gender,avg_wt
<chr>,<dbl>
Female,3203.117
Male,3279.816


#### Two grouping variables

Using `group_by` in the pipe

In [12]:
(births
 %>% group_by(Gender, Marital.Status)
 %>% summarize(avg_wt = mean(Birth.Weight.Grams,
                             na.rm = TRUE)
              )
 )

`summarise()` has grouped output by 'Gender'. You can override using the
`.groups` argument.


Gender,Marital.Status,avg_wt
<chr>,<chr>,<dbl>
Female,Married,3249.788
Female,Unmarried,3137.405
Male,Married,3321.245
Male,Unmarried,3219.448


Using `.by` argument in `summarise`

In [13]:
(births
 %>% summarize(avg_wt = mean(Birth.Weight.Grams,
                             na.rm = TRUE),
               .by = c(Gender, Marital.Status)
              )
 )

Gender,Marital.Status,avg_wt
<chr>,<chr>,<dbl>
Female,Unmarried,3137.405
Female,Married,3249.788
Male,Unmarried,3219.448
Male,Married,3321.245


## <font color="red"> Exercise 6.3.1 </font>

Use `group_by` and/or `summarise` to perform each of the following tasks.

**Tasks:**

1. Compute the overall average (`mean`) and standard deviation(`sd`) of the birth weights (`Birth.Weight.Grams`)
1. Compute the overall average (`mean`) and standard deviation(`sd`) of the birth weights (`Birth.Weight.Grams`) for each category of mother's smoking status (`Mother.Smoker`)

In [ ]:
# Your code here

## Working with a Categorical Response

Similar to the deficiencies we pointed out with JMP's **Table > Summary**,
`dplyr` makes us work when creating two way summary tables for two categorical variables.

### Getting category counts

It is relatively easy to get counts for a single or combination of categorical variables using the `count` function in the pipe. Note that this function performs a grouped aggregation.

In [16]:

(births
 %>% count(Low.Birth, Mother.Smoker)
 )

Low.Birth,Mother.Smoker,n
<chr>,<chr>,<int>
N,,7
N,N,1615
N,Y,203
Y,,1
Y,N,146
Y,Y,28


Alternatively, you can use `group_by` and `summarise` along with the `n` summary function, but this is approach results in needlessly complicated code.

In [17]:
(births
 %>% group_by(Low.Birth, Mother.Smoker)
 %>% summarize(cnt = n())
 )

`summarise()` has grouped output by 'Low.Birth'. You can override using the
`.groups` argument.


Low.Birth,Mother.Smoker,cnt
<chr>,<chr>,<int>
N,,7
N,N,1615
N,Y,203
Y,,1
Y,N,146
Y,Y,28


### Creating a two-way table with counts and row percents

Suppose that we want to summarize the relationship between an explanatory variable (Mother Smoker) and a response variable (Low Birth) in the usual way (explanatory variable in rows, summaries for the response variables in columns.  We can accomplish this by performing the following steps.

1. Determine all unique categories
2. Make indicator columns for each response category
3. Group by the explanatory variable and aggregate the response to get the counts by summing
4. Compute the row percents using the resulting counts

#### Step 1A - Determine the categories of the variables

In [19]:
births$Low.Birth %>% unique

[1] "N" "Y"

In [21]:
births$Mother.Smoker %>% unique

[1] "N" "Y" ""

#### Step 2A - Create indicator columns for each category of the response variable.

In [24]:
(births
 %>% select(Mother.Smoker, Low.Birth) # Temp. for convenience
 %>% mutate(low_birth = ifelse(Low.Birth=='Y',1,0),
            not_low_birth = ifelse(Low.Birth=='N',1,0)) # Indicators for the column labels
 %>% head # Temp. for convenience
)

,Mother.Smoker,Low.Birth,low_birth,not_low_birth
,<chr>,<chr>,<dbl>,<dbl>
1,N,N,0,1
2,Y,N,0,1
3,N,N,0,1
4,N,N,0,1
5,N,N,0,1
6,N,N,0,1


#### Step 3 - Compute the `sum` of each indicator

In [25]:
(births
 %>% mutate(low_birth = ifelse(Low.Birth=='Y',1,0),
            not_low_birth = ifelse(Low.Birth=='N',1,0))
 %>% group_by(Mother.Smoker) # Group by the row labels
 %>% summarise(total_low_birth = sum(low_birth),
               total_not_low_birth = sum(not_low_birth))
)

Mother.Smoker,total_low_birth,total_not_low_birth
<chr>,<dbl>,<dbl>
,1,7
N,146,1615
Y,28,203


#### Step 4 - Compute the row percents

In [26]:
(births
 %>% mutate(low_birth = ifelse(Low.Birth=='Y',1,0),
            not_low_birth = ifelse(Low.Birth=='N',1,0))
 %>% group_by(Mother.Smoker)
 %>% summarise(total_low_birth = sum(low_birth),
               total_not_low_birth = sum(not_low_birth))
 %>% mutate(percent_low_birth = round(total_low_birth/(total_low_birth + total_not_low_birth)*100, 1),
            percent_not_low_birth = round(total_not_low_birth/(total_low_birth + total_not_low_birth)*100, 1))
)

Mother.Smoker,total_low_birth,total_not_low_birth,percent_low_birth,percent_not_low_birth
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
,1,7,12.5,87.5
N,146,1615,8.3,91.7
Y,28,203,12.1,87.9


#### [Optional] Step 5 - Make `NA` more readable

In [27]:
(births
 %>% mutate(low_birth = ifelse(Low.Birth=='Y',1,0),
            not_low_birth = ifelse(Low.Birth=='N',1,0))
 %>% group_by(Mother.Smoker)
 %>% summarise(total_low_birth = sum(low_birth),
               total_not_low_birth = sum(not_low_birth))
 %>% mutate(percent_low_birth = round(total_low_birth/(total_low_birth + total_not_low_birth)*100, 1),
            percent_not_low_birth = round(total_not_low_birth/(total_low_birth + total_not_low_birth)*100, 1),
            Mother.Smoker = recode(Mother.Smoker,
                                   `Y` = 'Y',
                                   `N` = 'N',
                                   .default = "Unknown"))
)

Mother.Smoker,total_low_birth,total_not_low_birth,percent_low_birth,percent_not_low_birth
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
Unknown,1,7,12.5,87.5
N,146,1615,8.3,91.7
Y,28,203,12.1,87.9


## <font color="red"> Exercise 6.3.2 </font>

Use `group_by` and/or `summarise` to perform each of the following tasks.

**Tasks:**

1. Compute the counts (`n()`) for each category of the [Kotelchuck index](https://www-doh.state.nj.us/doh-shad/query/Kotelchuck.html) (`Kotelchuck.Index`)
2. Compute the counts and row percents when the explanatory variable is the Kotelchuck index (`Kotelchuck.Index`) and the response variable is whether or not there was a low birth weight (`Low.Birth`)

In [ ]:
# Your code here

***A mutate-group-aggregate-filter (note "" != NA):***

In [28]:
surveys <- read.csv('https://github.com/WSU-DataScience/DSCI_210_R_notebooks/raw/main/data/portal_data_joined.csv')
head(surveys)

,record_id,month,day,year,plot_id,species_id,sex,hindfoot_length,weight,genus,species,taxa,plot_type
,<int>,<int>,<int>,<int>,<int>,<chr>,<chr>,<int>,<int>,<chr>,<chr>,<chr>,<chr>
1,1,7,16,1977,2,NL,M,32,NA,Neotoma,albigula,Rodent,Control
2,72,8,19,1977,2,NL,M,31,NA,Neotoma,albigula,Rodent,Control
3,224,9,13,1977,2,NL,,NA,NA,Neotoma,albigula,Rodent,Control
4,266,10,16,1977,2,NL,,NA,NA,Neotoma,albigula,Rodent,Control
5,349,11,12,1977,2,NL,,NA,NA,Neotoma,albigula,Rodent,Control
6,363,11,12,1977,2,NL,,NA,NA,Neotoma,albigula,Rodent,Control


In [39]:
surveys$taxa %>% unique

[1] "Rodent"  "Rabbit"  "Bird"    "Reptile"

In [40]:
surveys %>%
  mutate(rodent = ifelse(taxa=='Rodent','yes','no')) %>%
  group_by(rodent, sex) %>%
  summarize(cnt = n(), avg_wt = mean(weight,na.rm=TRUE)) %>%
  filter(sex!="")

`summarise()` has grouped output by 'rodent'. You can override using the
`.groups` argument.


rodent,sex,cnt,avg_wt
<chr>,<chr>,<int>,<dbl>
yes,F,15690,42.17055
yes,M,17348,42.99538


# Translating JMP to `dplyr/tidyr`

### Simple Aggregations

```{r}
surveys %>%
  summarize(
    avg_wgt = mean(weight,
                   na.rm = TRUE))
```

<img width="850" src="https://raw.githubusercontent.com/WSU-DataScience/DSCI_210_R_notebooks/main/img/simple_aggregation_1.png">

<img width="850" src="https://raw.githubusercontent.com/WSU-DataScience/DSCI_210_R_notebooks/main/img/simple_aggregation_2.png">

### Group and Aggregate

```{r}
grp_agg <-
  surveys %>%
    group_by(taxa) %>%
    summarize(avg = mean(weight, na.rm = TRUE))
grp_agg
```

<img width="850" src="https://raw.githubusercontent.com/WSU-DataScience/DSCI_210_R_notebooks/main/img/group_and_aggregate_0.png">

<img width="850" src="https://raw.githubusercontent.com/WSU-DataScience/DSCI_210_R_notebooks/main/img/group_and_aggregate_1.png">

<img width="850" src="https://raw.githubusercontent.com/WSU-DataScience/DSCI_210_R_notebooks/main/img/group_and_aggregate_2.png">

### Multiple Group Variables

<img width="850" src="https://raw.githubusercontent.com/WSU-DataScience/DSCI_210_R_notebooks/main/img/grp2_agg.PNG">